# Setup

In [ ]:
import pandapower.networks as pnet
import pandas as pd
import numpy as np
import scipy as sp
import cvxpy as cp
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, '..')

from src.simulation.noise import add_noise_in_cartesian_coordinates
from src.models.regression import ComplexRegression, ComplexLasso
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import error_metrics, fro_error

# Network simulation

In [ ]:
net = pnet.create_kerber_landnetz_freileitung_2()
nodes = net.bus.shape[0]
steps = 400
load_cv = 0.2
current_accuracy = 0.0001
voltage_accuracy = 0.0001

In [ ]:
np.random.seed(11)
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, steps)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)
controlled_net.bus

In [ ]:
voltage = voltage[:, 1:]
y_bus = y_bus[1:, 1:]
current = np.array(voltage @ y_bus)
noisy_voltage, noisy_current = add_noise_in_cartesian_coordinates(current, voltage, current_accuracy, voltage_accuracy)
voltage_error, current_error = noisy_voltage - voltage, noisy_current - current

In [ ]:
np.linalg.svd(voltage, compute_uv=False)

# OLS Identification

In [ ]:
ols = ComplexRegression()
ols.fit(noisy_voltage, noisy_current)
y_ols = ols.fitted_admittance_matrix
error_metrics(y_bus, y_ols)

# Lasso Identification

In [ ]:
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-12, -8, 20), solver=cp.GUROBI)
lasso.fit(noisy_voltage, noisy_current)
y_lasso = lasso.fitted_admittance_matrix

In [ ]:
lasso.best_trial

In [ ]:
error_metrics(y_bus, y_lasso)

# TLS Identification

In [ ]:
%%time
eiv = TotalLeastSquares()
eiv.fit(noisy_voltage, noisy_current)
y_tls = eiv.fitted_admittance_matrix

In [ ]:
error_metrics(y_bus, y_tls)

# L1 Regularized TLS

In [ ]:
tls_lambda = 5e-10 # lasso.best_hyperparams['lambda']
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=10e-12, rel_tol=10e-12, max_iterations=500, verbose=False, solver=cp.GUROBI)
sparse_eiv.fit(noisy_voltage, noisy_current)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix

In [ ]:
error_metrics(y_bus, y_sparse_eiv)

In [ ]:
y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_eiv.iterations])
pd.Series(y_errors).plot(title='Fro error on Y')

In [ ]:
targets = pd.Series([i.target_function for i in sparse_eiv.iterations])
pd.Series(targets).plot(title='Target function')

# Result analysis

In [ ]:
sns.heatmap(np.abs(y_bus));

In [ ]:
sns.heatmap(np.abs(y_bus - y_ols));

In [ ]:
sns.heatmap(np.abs(y_bus - y_lasso));

In [ ]:
sns.heatmap(np.abs(y_bus - y_tls));

In [ ]:
sns.heatmap(np.abs(y_bus - y_sparse_eiv));

In [ ]:
y_bus

In [ ]:
print(f'Norm of current error: {np.linalg.norm(current_error, "fro")**2}')
print(f'Norm of admittance matrix: {np.sum(np.abs(np.real(y_bus))) + np.sum(np.abs(np.imag(y_bus)))}')